In [1]:
# task2.4. Реализовать на базе модели определения пола выявление с другим нормированием (относительно среднего значения). Также добавьте альтернативную функцию активации
# task2.5. Реализовать решение уровнения через градиентный спуск: x**2+2x-6
# task2.6. (дополнительно) Реализовать решение уровнения через градиентный спуск: x**3+2x**2-6x-1